In [1]:
include("lib/all.jl");

In [2]:
test_mov()

706345

In [3]:
test_acc()

839979

In [4]:
particles = make_particles(2^20);
t = Tree(particles, 16);

In [5]:
@time group!(t);

In [6]:
@time inform!(t);

  0.184846 seconds (881.23 k allocations: 14.678 MB, 30.70% gc time)


In [7]:
test_in_cell_mass(t)

In [8]:
ax = zeros(length(particles))
ay = zeros(length(particles))
az = zeros(length(particles));

In [11]:
@time interact!(t, 0.5, ax, ay, az);

  1.649739 seconds (4 allocations: 160 bytes)
